In [1]:
from collections import namedtuple
import inspect

from lib import promotions

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

promos = [func for name, func in
                inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

In [2]:
#顧客
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

#購物車
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [3]:
Order(joe, cart , best_promo)

<Order total: 42.00 due: 42.00>

In [4]:
Order(ann, cart , best_promo)

<Order total: 42.00 due: 39.90>

In [5]:
Order(ann, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [6]:
Order(ann, long_order, best_promo)

<Order total: 10.00 due: 9.30>